# JOIN

Join is a keyword that allows you to combine information from different tables. It's really important, but sometimes it's hard to understand, so this whole page is about features and usecases of joins in Postgres SQL.

In the following cell, I'm creating a database that will be used for all the examples on this page.

In [1]:
%%bash
docker run -d --rm\
    --name join_example\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5
docker exec -i join_example psql -U postgres -d postgres

CREATE TABLE central_table (
    id INT NOT NULL,
    someinfo1_id INT NOT NULL,
    someinfo2_id INT NOT NULL
);
INSERT INTO central_table (id, someinfo1_id, someinfo2_id) VALUES
(1, 1, 1),
(2, 2, 2),
(3, 2, 1);

CREATE TABLE some_info1 (
    id INT NOT NULL,
    some_info TEXT
);
INSERT INTO some_info1 (id, some_info) VALUES
(1, 'some_info1.1'),
(2, 'some_info1.2'),
(3, 'some_info1.3');

CREATE TABLE some_info2 (
    id INT NOT NULL,
    some_info TEXT
);
INSERT INTO some_info2 (id, some_info) VALUES
(1, 'some_info2.1'),
(2, 'some_info2.2'),
(3, 'some_info2.3');

CREATE TABLE
INSERT 0 3
CREATE TABLE
INSERT 0 3
CREATE TABLE
INSERT 0 3


In basic case examples I'll use tables `central_table`, `some_info1`, `some_info2`. So in the next cell I just show their content.

In [75]:
%%bash
docker exec -i join_example psql -U postgres -d postgres

SELECT * FROM central_table;
SELECT * FROM some_info1;
SELECT * FROM some_info2;

 id | someinfo1_id | someinfo2_id 
----+--------------+--------------
  1 |            1 |            1
  2 |            2 |            2
  3 |            2 |            1
(3 rows)

 id |  some_info   
----+--------------
  1 | some_info1.1
  2 | some_info1.2
  3 | some_info1.3
(3 rows)

 id |  some_info   
----+--------------
  1 | some_info2.1
  2 | some_info2.2
  3 | some_info2.3
(3 rows)



**Note** Remember to close the container when you have finished playing with the examples.

In [73]:
!docker stop join_example

join_example


## Short names

You can use short names for the tables involved in the join to make it easier to write code. Just give the new name of the table separated by a space from the original name of the table.

In the next cell I set the `sent` name for the table `central_table` and the `si1` name for the table `some_info1` and use it everywhere to refer to the tables used in the join.

In [77]:
%%bash
docker exec -i join_example psql -U postgres -d postgres

SELECT sent.id, si1.some_info FROM 
    central_table sent
    JOIN
    some_info1 si1
    ON sent.someinfo1_id=si1.id
;

 id |  some_info   
----+--------------
  1 | some_info1.1
  2 | some_info1.2
  3 | some_info1.2
(3 rows)



## More tables

You can use more than two tables in a join. Just use multiple `JOIN ... ON ...` blocks to select more information.

So in the following example, I simply combine all the sample database information into one query!

In [79]:
%%bash
docker exec -i join_example psql -U postgres -d postgres

SELECT sent.id, si1.some_info, si2.some_info FROM 
    central_table sent
    JOIN
    some_info1 si1
    ON sent.someinfo1_id=si1.id
    JOIN
    some_info2 si2
    ON sent.someinfo2_id=si2.id
;

 id |  some_info   |  some_info   
----+--------------+--------------
  1 | some_info1.1 | some_info2.1
  3 | some_info1.2 | some_info2.1
  2 | some_info1.2 | some_info2.2
(3 rows)



## Duplication

If in joined table will be duplicates in join key, you will find that some tables of original table have been duplicated.

In the example for this page, `central_table.someinfo1_id` takes the value `2` twice. So if you join it to `some_info`, the record of `some_info` that has `id=2` will be duplicated, just like in the following cell.

In [4]:
%%bash
docker exec -i join_example psql -U postgres -d postgres

SELECT * 
FROM some_info1
FULL OUTER JOIN central_table ON some_info1.id=central_table.someinfo1_id;

 id |  some_info   | id | someinfo1_id | someinfo2_id 
----+--------------+----+--------------+--------------
  1 | some_info1.1 |  1 |            1 |            1
  2 | some_info1.2 |  2 |            2 |            2
  2 | some_info1.2 |  3 |            2 |            1
  3 | some_info1.3 |    |              |             
(4 rows)



## Join types

This diagram describes the different types of joins available in SQL.

<img src="join_files/join_types.png"></img>